In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import utils

In [2]:
rows = ['Zonal_Winds', 'Meridional_Winds', 'Humidity', 'Air_Temp', 'Sea_Surface_Temp']
train_data = pd.read_csv('../../Cleaning-the-data/tao-cleaned.csv')[rows]
train_label = train_data['Sea_Surface_Temp']

train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:15000]
train_label = train_label.values[:15000]
train_data.shape

(15000, 5)

In [3]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(5,32)
        self.lstm2 = nn.LSTM(32,32)
        self.lstm3 = nn.LSTM(32,1)
 
    def forward(self, seq):
        lstm1_out, _ = self.lstm1(seq, None)
        lstm2_out, _ = self.lstm2(lstm1_out, None)
        lstm3_out, _ = self.lstm3(lstm2_out, None)
        return lstm3_out.squeeze(1)

In [4]:
seq = Sequence()
test_data = torch.Tensor(train_data[:10]).view(-1,1,5)
test_label = torch.Tensor(train_data[:10][:,-1]).view(-1,1)

print(test_label)
print(seq(test_data))

tensor([[-0.7540],
        [-0.8920],
        [-1.2446],
        [-1.7403],
        [-2.3127],
        [-2.0827],
        [-1.7250],
        [-1.8272],
        [-1.7761],
        [-0.9073]])
tensor([[-0.2751],
        [-0.2829],
        [-0.2654],
        [-0.2554],
        [-0.2500],
        [-0.2476],
        [-0.2463],
        [-0.2446],
        [-0.2425],
        [-0.2404]], grad_fn=<SqueezeBackward1>)


In [5]:
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.01)
seq_size = 50

In [6]:
for epoch in range(50):
    running_loss = 0
    running_error = 0
    num_batches = 0
    for i in range(0, 15000, seq_size):
        data = train_data[i:i+seq_size]
        xs = torch.Tensor(data[:-1]).view(-1, 1, 5)
        ys = torch.Tensor(data[1:][:,-1]).view(-1,1)
        
        lstm_out = seq(xs)
        loss = criterion(lstm_out.view(1, 49), ys.view(1, 49))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute some stats:
        
        num_batches+=1
        
        with torch.no_grad():
            running_loss += loss.item()
            error = utils.get_error(lstm_out.view(1, 49), ys.view(1, 49))
            running_error += error 

    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    if epoch % 1 == 0:
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches

        print('epoch=',epoch, '\t loss=', total_loss, '\t error={:2.2%}'.format(total_error) ,'percent')

epoch= 0 	 loss= 1.1407398899415664 	 error=84.56% percent
epoch= 1 	 loss= 1.0392989963506503 	 error=79.90% percent
epoch= 2 	 loss= 1.0301013463142348 	 error=75.31% percent
epoch= 3 	 loss= 1.024320229133967 	 error=71.76% percent
epoch= 4 	 loss= 1.0232329306117753 	 error=70.82% percent
epoch= 5 	 loss= 1.0245121108239497 	 error=72.62% percent
epoch= 6 	 loss= 1.0222426898332682 	 error=69.44% percent
epoch= 7 	 loss= 1.0216932713252511 	 error=69.90% percent
epoch= 8 	 loss= 1.0208474901962472 	 error=68.52% percent
epoch= 9 	 loss= 1.0211938668051517 	 error=68.90% percent
epoch= 10 	 loss= 1.0204881660733403 	 error=68.66% percent
epoch= 11 	 loss= 1.0197805684352959 	 error=67.57% percent
epoch= 12 	 loss= 1.0200029189843554 	 error=68.05% percent
epoch= 13 	 loss= 1.0219034061943604 	 error=72.14% percent
epoch= 14 	 loss= 1.0241992912491773 	 error=73.12% percent
epoch= 15 	 loss= 1.0210054310877907 	 error=69.52% percent
epoch= 16 	 loss= 1.0204321324747858 	 error=68.49%